# Assignment 1: Data Parsing, Cleansing and Integration
## Task 3
#### Student Name: Ricky Truong
#### Student ID: s3783560

Date: 26/04/2024

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy


## Introduction
Give general information of this assessment here, e.g., what problems you encounter in integration, general steps and solutions you've taken to integrate the data. 

This notebook, explores the problem of integration, where we integrated the first dataset obtain from task 1 and 2 with a second dataset in the form of a csv. During the integration task, we encountered the problem of semantic and schema conflicts. Where the schema in dataset2 did not match the global schema which meant we have to revise the schema in the dataset. This included adding new columns with imputate values as well as renaming columns to fit the global schema. Additionally, there was the problem of data conflicts where sometimes the data values did not match the allowed values in the global schema. For example, the salary column did not reflect annual salary but rather monthly salary. This was fix via a formuala to calculate annual salary from monthly salary. Another example was found in the ContractType, where it had different values such as F.T.E 0.2, which was different from the allowed values in that particular column. 

After resolving all these conflicts we then merged the two datasets, where we encoutered the problem of duplicates, which we only kept the first duplicated and deleleted the second version.


##  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment
import pandas as pd
import numpy as np
import xml.etree.ElementTree as etree
from itertools import chain
import re 
import difflib

### 1. Examining and loading data
Examine "\<student\_id\>_dataset2.csv" the structure and schema. 

In [2]:
# Code to inspect the provided data file
# Couple of notes for all code block in this notebook
## please provide proper comment on your code
## Please re-start and run all cells to make sure codes are runable and include your output in the submission
df2 = pd.read_csv('s3783560_dataset2.csv')
df2.head()

,Job Title,Organisation,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening
0,**** Clinical Nurse Advisors York York,Hays Healthcare,1916.67,NaN,Healthcare and Nursing,26/06/2013 0:00,York,FULL TIME,28/03/2013 0:00
1,Scrub Nurse Maidstone,Compass Associates Ltd,2500.00,NaN,Healthcare and Nursing,3/05/2013 15:00,Maidstone,FULL TIME,2/02/2013 15:00
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,1916.67,NaN,Healthcare and Nursing,1/05/2012 0:00,Bedford,FULL TIME,1/04/2012 0:00
3,Full Or Part Time Outreach Support Taunton,BS Social Care,1099.58,NaN,Healthcare and Nursing,7/05/2013 0:00,Taunton,FULL TIME,7/04/2013 0:00
4,Registered General Nurse Hertford,TXM Healthcare,2125.00,NaN,Healthcare and Nursing,15/10/2012 15:00,Hertford,FULL TIME,16/08/2012 15:00


### 2. Resolving schema conflicts
Most of the conflicted we found where in relation to the global schema where dataset2 did not match the global scheme provided. For example, names of columns were incorrect as well as how the data was represented in salary. Salary was represented as a monthly payment instead of annual salary, like in dataset1.
 
Further information on the conflict and how we resolved them is provided in the appropriately sections.

#### Conflict 1: Map Sources to Gloabl Schema (e.g. naming conflicts in Job Title, Organisation, Closing, Opening)
The following column names do not match the the global schema.


In [3]:
# Code to inspect data and identify conflicts in schema
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Job Title                   5000 non-null   object 
 1   Organisation                4937 non-null   object 
 2   Monthly Payment             5000 non-null   float64
 3   Contract Type               4312 non-null   object 
 4   Category                    5000 non-null   object 
 5   Closing                     5000 non-null   object 
 6   Location                    5000 non-null   object 
 7   Full-Time Equivalent (FTE)  643 non-null    object 
 8   Opening                     5000 non-null   object 
dtypes: float64(1), object(8)
memory usage: 351.7+ KB



To rename the columns we use the rename function while passing in a dictionary that will the column name to a new name

In [4]:
# Code to fix the conflict
df2 = df2.rename(columns = {'Job Title': 'Title', 'Organisation':'Company','Closing': 'CloseDate', 'Opening': 'OpenDate'})
df2.head()

,Title,Company,Monthly Payment,Contract Type,Category,CloseDate,Location,Full-Time Equivalent (FTE),OpenDate
0,**** Clinical Nurse Advisors York York,Hays Healthcare,1916.67,NaN,Healthcare and Nursing,26/06/2013 0:00,York,FULL TIME,28/03/2013 0:00
1,Scrub Nurse Maidstone,Compass Associates Ltd,2500.00,NaN,Healthcare and Nursing,3/05/2013 15:00,Maidstone,FULL TIME,2/02/2013 15:00
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,1916.67,NaN,Healthcare and Nursing,1/05/2012 0:00,Bedford,FULL TIME,1/04/2012 0:00
3,Full Or Part Time Outreach Support Taunton,BS Social Care,1099.58,NaN,Healthcare and Nursing,7/05/2013 0:00,Taunton,FULL TIME,7/04/2013 0:00
4,Registered General Nurse Hertford,TXM Healthcare,2125.00,NaN,Healthcare and Nursing,15/10/2012 15:00,Hertford,FULL TIME,16/08/2012 15:00


#### Conflict 2:  Map Sources to Gloabl Schema (Full Time Equivalent (FTE))
The column 'Full Time Equivalent (FTE) is the equivalent to the the column ContractType in the global schema'. This means the values of the column can only be 'full_time', 'part_time' or 'non'specific'.

In [5]:
#inspect all possible values
df2['Full-Time Equivalent (FTE)'].value_counts()

FULL TIME    583
0.2 FTE       16
0.4 FTE       15
0.6 FTE       15
0.8 FTE       14
Name: Full-Time Equivalent (FTE), dtype: int64

From what we can see the values in the column 'Full-Time Equivalent (FTE)' does not match the the global schema. To fix this, all cells with the value 'FULL TIME' will be replaced with 'full_time', while the rest of the remaning values, for example 0.2 FTE and 0.8 FTE will be mapped to 'part_time' as these values suggest that they do not work the full 'full_time' hours. Additionally the column name will be changed to ContractType to reflect the values in these columns

In [6]:
#map FULL TIME to full_time
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x:'full_time' if x == 'FULL TIME' else x)

#map remaining values to part time e.g 0.2 FTE and 0.4 FTE
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x:'part_time' if x == '0.2 FTE' else x)
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x:'part_time' if x == '0.4 FTE' else x)
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x:'part_time' if x == '0.6 FTE' else x)
df2['Full-Time Equivalent (FTE)'] = df2['Full-Time Equivalent (FTE)'].apply(lambda x:'part_time' if x == '0.8 FTE' else x)

#rename column
df2 = df2.rename(columns = {'Full-Time Equivalent (FTE)': 'ContractType'})

#confirm the values
df2['ContractType'].value_counts()

full_time    583
part_time     60
Name: ContractType, dtype: int64

#### Conflict 3:  Map Sources to Gloabl Schema (Contract Type)
The column 'Contract Type' is equivalent to the global schema column 'ContractTime'. This means all possible values for this column will be either 'permanent', 'contract' or 'non-specific'


In [7]:
#see all possible values for contract type
df2['Contract Type'].value_counts()

Permanent              3720
Fixed Term Contract     592
Name: Contract Type, dtype: int64

From the values above we can assume that any cell with the values 'Permanent' will be mapped to 'permanent', while 'Fix Term Contract' will be mapped to 'contract'. Missing values will be dealt with later. This mapping will be done with the apply  and lambda function.

In [8]:
# map cells values to their appropriate values using .apply()
df2['Contract Type'] = df2['Contract Type'].apply(lambda x:'permanent' if x == 'Permanent' else x)
df2['Contract Type'] = df2['Contract Type'].apply(lambda x:'contract' if x == 'Fixed Term Contract' else x)

#check values for change
print(df2['Contract Type'].value_counts())

#rename column
df2 = df2.rename(columns = {'Contract Type': 'ContractTime'})

permanent    3720
contract      592
Name: Contract Type, dtype: int64


#### Conflict 4:  Map Sources to Gloabl Schema (Monthly Payment)
The column 'Monthly Payment' is equivalent to the 'Salary' column in the the global schema, but the values to do not contain the yearly salary but instead the month salary.

In [9]:
df2['Monthly Payment']

0       1916.67
1       2500.00
2       1916.67
3       1099.58
4       2125.00
         ...   
4995    1916.67
4996    1875.00
4997    2708.33
4998     417.17
4999    1900.00
Name: Monthly Payment, Length: 5000, dtype: float64

There are 12 months in a year, therefore to convert this column to an annual salary we will multiply all values by 12 and that will be the new value of the cell. We will skip all NULL values as that will be dealt with later on.

In [10]:
#multiply every non null value by 12
df2['Monthly Payment'] = df2['Monthly Payment'].apply(lambda x: x*12 if x != None else x)

#change Monthly Payment to Salary
df2 = df2.rename(columns = {'Monthly Payment': 'Salary'})

#### Conflict 5:  Map Sources to Gloabl Schema (OpenDate and CloseDate)
The datetime formats do not follow the global schema of yyyy-mm-dd hh:mm:ss, but instead follow the format dd/mm/yyyy hh:mm

In [11]:
df2[['CloseDate','OpenDate']]

,CloseDate,OpenDate
0,26/06/2013 0:00,28/03/2013 0:00
1,3/05/2013 15:00,2/02/2013 15:00
2,1/05/2012 0:00,1/04/2012 0:00
3,7/05/2013 0:00,7/04/2013 0:00
4,15/10/2012 15:00,16/08/2012 15:00
...,...,...
4995,24/10/2013 15:00,24/09/2013 15:00
4996,20/06/2012 12:00,21/05/2012 12:00
4997,17/12/2013 15:00,17/11/2013 15:00
4998,15/03/2012 12:00,15/01/2012 12:00


To fix this we convert the columns to datetime datatype first, before converting the time date into the required format of yyyy-mm-dd hh:mm:ss. Since there are no seconds we assume '00' for the seconds

In [12]:
#convert OpenDate to appropriate format
df2['OpenDate'] = pd.to_datetime(df2['OpenDate']) # convert object to datetime obj
df2['OpenDate'] = df2['OpenDate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')) # convert to required format

In [13]:
#convert CloseDate to appropriate format
df2['CloseDate'] = pd.to_datetime(df2['CloseDate']) # convert object to datetime obj
df2['CloseDate'] = df2['CloseDate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')) # convert to required format

In [14]:
#confirm change
df2.head(3)

,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate
0,**** Clinical Nurse Advisors York York,Hays Healthcare,23000.04,NaN,Healthcare and Nursing,2013-06-26 00:00:00,York,full_time,2013-03-28 00:00:00
1,Scrub Nurse Maidstone,Compass Associates Ltd,30000.00,NaN,Healthcare and Nursing,2013-03-05 15:00:00,Maidstone,full_time,2013-02-02 15:00:00
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,23000.04,NaN,Healthcare and Nursing,2012-01-05 00:00:00,Bedford,full_time,2012-01-04 00:00:00


#### Conflict 6:  Map Sources to Gloabl Schema (Category)
Values in the Category column do not match the global schema where there are 8 possibilites are ‘IT Jobs’, ‘Healthcare & Nursing Jobs’,‘Engineering Jobs’, ‘Accounting & Finance Jobs’, ‘Sales Jobs’, ‘Hospitality & Catering
Jobs’, ‘Teaching Jobs’, ‘PR, Advertising & Marketing Jobs’.

In [15]:
df2['Category'].value_counts()

Information Technology           1286
Engineering                      1066
Healthcare and Nursing            655
Finance and Accounting            637
Sales                             630
Hospitality and Catering          275
Teaching                          234
PR, Advertising and Marketing     217
Name: Category, dtype: int64

Under the assumption that IT is short for Information Technology, all values are the missing the additionally string of Jobs at the end. To do this we simply use the .apply with a lambda function inside.

In [16]:
# Convert Information Technology first to make it easier to do the rest
df2['Category'] = df2['Category'].apply(lambda x: 'IT Jobs' if x == 'Information Technology' else x)

#add string ' Jobs' to every value except when value= 'IT Jobs'
df2['Category'] = df2['Category'].apply(lambda x: x + ' Jobs' if x != 'IT Jobs' else x)

#confirm change
df2['Category'].value_counts()

IT Jobs                               1286
Engineering Jobs                      1066
Healthcare and Nursing Jobs            655
Finance and Accounting Jobs            637
Sales Jobs                             630
Hospitality and Catering Jobs          275
Teaching Jobs                          234
PR, Advertising and Marketing Jobs     217
Name: Category, dtype: int64

In [17]:
df2.head()

,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate
0,**** Clinical Nurse Advisors York York,Hays Healthcare,23000.04,NaN,Healthcare and Nursing Jobs,2013-06-26 00:00:00,York,full_time,2013-03-28 00:00:00
1,Scrub Nurse Maidstone,Compass Associates Ltd,30000.00,NaN,Healthcare and Nursing Jobs,2013-03-05 15:00:00,Maidstone,full_time,2013-02-02 15:00:00
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,23000.04,NaN,Healthcare and Nursing Jobs,2012-01-05 00:00:00,Bedford,full_time,2012-01-04 00:00:00
3,Full Or Part Time Outreach Support Taunton,BS Social Care,13194.96,NaN,Healthcare and Nursing Jobs,2013-07-05 00:00:00,Taunton,full_time,2013-07-04 00:00:00
4,Registered General Nurse Hertford,TXM Healthcare,25500.00,NaN,Healthcare and Nursing Jobs,2012-10-15 15:00:00,Hertford,full_time,2012-08-16 15:00:00


#### Conflict 7: Map Sources to Gloabl Schema (Missing Data)
When we first explored the dataset, we discovered some cell values contained NULL or None indicating missing data. For missing data in the global schema, all string data types would be allocated the value 'non-specific' while all numerica data types would be allocated the value 0.00

In [18]:
df2.info()
print(df2.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         5000 non-null   object 
 1   Company       4937 non-null   object 
 2   Salary        5000 non-null   float64
 3   ContractTime  4312 non-null   object 
 4   Category      5000 non-null   object 
 5   CloseDate     5000 non-null   object 
 6   Location      5000 non-null   object 
 7   ContractType  643 non-null    object 
 8   OpenDate      5000 non-null   object 
dtypes: float64(1), object(8)
memory usage: 351.7+ KB
(5000, 9)


In [19]:
df2.isnull().sum()

Title              0
Company           63
Salary             0
ContractTime     688
Category           0
CloseDate          0
Location           0
ContractType    4357
OpenDate           0
dtype: int64

From the above stats we see that the total number of instaces should equal 5000. However in Company, ContractTime and Contract type there exist missing data. All of these are strings datatypes, hence they will be replaced with 'non-specific' using the .fillna function.

In [20]:
df2 = df2.fillna({'Company': 'non-specific','Salary':'0.00', 'ContractTime': 'non-specific', 'ContractType': 'non-specific'})
df2.isnull().sum()

Title           0
Company         0
Salary          0
ContractTime    0
Category        0
CloseDate       0
Location        0
ContractType    0
OpenDate        0
dtype: int64

In [21]:
df2.head()

,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate
0,**** Clinical Nurse Advisors York York,Hays Healthcare,23000.04,non-specific,Healthcare and Nursing Jobs,2013-06-26 00:00:00,York,full_time,2013-03-28 00:00:00
1,Scrub Nurse Maidstone,Compass Associates Ltd,30000.00,non-specific,Healthcare and Nursing Jobs,2013-03-05 15:00:00,Maidstone,full_time,2013-02-02 15:00:00
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,23000.04,non-specific,Healthcare and Nursing Jobs,2012-01-05 00:00:00,Bedford,full_time,2012-01-04 00:00:00
3,Full Or Part Time Outreach Support Taunton,BS Social Care,13194.96,non-specific,Healthcare and Nursing Jobs,2013-07-05 00:00:00,Taunton,full_time,2013-07-04 00:00:00
4,Registered General Nurse Hertford,TXM Healthcare,25500.00,non-specific,Healthcare and Nursing Jobs,2012-10-15 15:00:00,Hertford,full_time,2012-08-16 15:00:00


#### Conflict 8: Map Sources to Global Schema (Missing columns)

According to the global schema, we are missing the columns "Id" and "SourceName". The "SourceName" column will all be imputated with the value 'non-specific' as there is no way to determine where the sources of the jobs came from. To resolve the missing 'Id' column we will number there job at the first index as 1 and so on until all jobs are numbered.

In [22]:
id_list = []
for i in range(1,5001): # start from 1 and go to 5000
    id_list.append(str(i)) # creates a list of digits from 1-5000
df2 = df2.assign(Id = id_list) # assign column 'Id' with list of digits
df2['Id'] = df2['Id'].str.zfill(8) # pad with 0 for 8 digit int

source_name = []
for i in range(5000):
    source_name.append('non-specific') #append non-specific 5000 times

df2 = df2.assign(SourceName = source_name) # create column SourceName with values of source.

    

In [23]:
print(df2.shape)
df2.head()

(5000, 11)


,Title,Company,Salary,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Id,SourceName
0,**** Clinical Nurse Advisors York York,Hays Healthcare,23000.04,non-specific,Healthcare and Nursing Jobs,2013-06-26 00:00:00,York,full_time,2013-03-28 00:00:00,00000001,non-specific
1,Scrub Nurse Maidstone,Compass Associates Ltd,30000.00,non-specific,Healthcare and Nursing Jobs,2013-03-05 15:00:00,Maidstone,full_time,2013-02-02 15:00:00,00000002,non-specific
2,Clinical Nurse Advisor Bedford Bedford,Hays Healthcare,23000.04,non-specific,Healthcare and Nursing Jobs,2012-01-05 00:00:00,Bedford,full_time,2012-01-04 00:00:00,00000003,non-specific
3,Full Or Part Time Outreach Support Taunton,BS Social Care,13194.96,non-specific,Healthcare and Nursing Jobs,2013-07-05 00:00:00,Taunton,full_time,2013-07-04 00:00:00,00000004,non-specific
4,Registered General Nurse Hertford,TXM Healthcare,25500.00,non-specific,Healthcare and Nursing Jobs,2012-10-15 15:00:00,Hertford,full_time,2012-08-16 15:00:00,00000005,non-specific


In [24]:

df1 = pd.read_csv('s3783560_dataset1_solution.csv')
#convert data type in df2 to df1 datatypes
for x in [c for c in df1.columns if c in df2.columns]:
    if df1[x].dtype !=df2[x].dtype:
        df2[x] = df2[x].astype(df1[x].dtype)


In [25]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50703 entries, 0 to 50702
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            50703 non-null  int64  
 1   Title         50703 non-null  object 
 2   Location      50703 non-null  object 
 3   Company       50703 non-null  object 
 4   Category      50703 non-null  object 
 5   Salary        50703 non-null  float64
 6   OpenDate      50703 non-null  object 
 7   CloseDate     50703 non-null  object 
 8   ContractTime  50703 non-null  object 
 9   SourceName    50703 non-null  object 
 10  ContractType  50703 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.3+ MB


In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         5000 non-null   object 
 1   Company       5000 non-null   object 
 2   Salary        5000 non-null   float64
 3   ContractTime  5000 non-null   object 
 4   Category      5000 non-null   object 
 5   CloseDate     5000 non-null   object 
 6   Location      5000 non-null   object 
 7   ContractType  5000 non-null   object 
 8   OpenDate      5000 non-null   object 
 9   Id            5000 non-null   int64  
 10  SourceName    5000 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 429.8+ KB


### 3. Merging data

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [27]:
# Code to merge two data sets
df = pd.concat([df1,df2])
df.head()

,Id,Title,Location,Company,Category,Salary,OpenDate,CloseDate,ContractTime,SourceName,ContractType
0,69249311,Development Management Team Manager,North West England,Matchtech Group plc.,Engineering Jobs,41000.0,2013-11-18 12:00:00,2014-01-17 12:00:00,non-specific,rengineeringjobs.com,non-specific
1,70676070,Senior Civil Infrastructure Engineer,Basingstoke,Matchtech Group plc.,Engineering Jobs,32000.0,2013-06-02 15:00:00,2013-06-16 15:00:00,non-specific,rengineeringjobs.com,non-specific
2,68805528,Highway Technician,Uk,Matchtech Group plc.,Engineering Jobs,19500.0,2012-12-24 15:00:00,2013-01-23 15:00:00,non-specific,rengineeringjobs.com,non-specific
3,71684871,Combustion Engineer Surrey And London upto ****k,London,Randstad,Engineering Jobs,33500.0,2012-10-19 12:00:00,2013-01-17 12:00:00,permanent,rengineeringjobs.com,non-specific
4,67944676,Automation Service Manager,Stockton-On-Tees,Matchtech Group plc.,Engineering Jobs,45000.0,2012-05-03 12:00:00,2012-05-17 12:00:00,non-specific,rengineeringjobs.com,non-specific


### 4. Resolving data conflicts:
Give some general information about the data conflicts you found in the unified table and how you resolve them.

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

#### Conflict 1: XXXX (e.g., complete duplications)
Here give some general information about this conflict.

In [28]:
# Code to inspect data and identify conflicts in data
#see dupes based on title location opendate and closedate as these will be the same job
cols = ['Title','Location','OpenDate','CloseDate']
duplicates = df[df.duplicated(cols,keep= False)]
print(duplicates.shape)
duplicates.sort_values(by = cols).head(10)

(238, 11)


,Id,Title,Location,Company,Category,Salary,OpenDate,CloseDate,ContractTime,SourceName,ContractType
3212,72479948,3rd Line Support Network Wireless,Nottingham,Hays Corporate Accounts,IT Jobs,42500.00,2013-09-16 12:00:00,2013-11-15 12:00:00,permanent,jobsite.co.uk,non-specific
3240,3241,3rd Line Support Network Wireless,Nottingham,Hays Corporate Accounts,IT Jobs,42500.04,2013-09-16 12:00:00,2013-11-15 12:00:00,permanent,non-specific,non-specific
42125,72484204,ASPNet Developer / PERM / ****k,Cannock,Computer Futures,IT Jobs,40000.00,2013-04-28 15:00:00,2013-05-28 15:00:00,non-specific,gojobsearch.co.uk,non-specific
3365,3366,ASPNet Developer / PERM / ****k,Cannock,Computer Futures,IT Jobs,39999.96,2013-04-28 15:00:00,2013-05-28 15:00:00,non-specific,non-specific,non-specific
45217,72192876,ASPnet C Snr Developer,Hampshire,non-specific,IT Jobs,44000.00,2012-02-26 00:00:00,2012-05-26 00:00:00,permanent,jobs.telegraph.co.uk,non-specific
38,39,ASPnet C Snr Developer,Hampshire,non-specific,IT Jobs,44000.04,2012-02-26 00:00:00,2012-05-26 00:00:00,permanent,non-specific,non-specific
44739,72385280,Account Director Digital Marketing,Leicestershire,non-specific,"PR, Advertising & Marketing Jobs",44000.00,2013-12-19 00:00:00,2014-02-17 00:00:00,non-specific,onlymarketingjobs.com,non-specific
1524,1525,Account Director Digital Marketing,Leicestershire,non-specific,"PR, Advertising and Marketing Jobs",44000.04,2013-12-19 00:00:00,2014-02-17 00:00:00,non-specific,non-specific,non-specific
16019,72464811,Account Director – BTL / Promotional & Retail ...,Central London,Fill Recruitment Ltd,"PR, Advertising & Marketing Jobs",45000.00,2012-11-18 12:00:00,2012-12-18 12:00:00,permanent,Brand Republic Jobs,full_time
3181,3182,Account Director – BTL / Promotional & Retail ...,Central London,Fill Recruitment Ltd,"PR, Advertising and Marketing Jobs",45000.00,2012-11-18 12:00:00,2012-12-18 12:00:00,permanent,non-specific,full_time


To fix the duplication here we will keep the first copy as the second copy will contain mostly missing values or imputated values

In [29]:
# Code to fix the conflict
#keep the first duplicate and delete the second 
df = df.drop_duplicates(cols, keep = 'first')
print(df.shape)
df.head(10)

(55584, 11)


,Id,Title,Location,Company,Category,Salary,OpenDate,CloseDate,ContractTime,SourceName,ContractType
0,69249311,Development Management Team Manager,North West England,Matchtech Group plc.,Engineering Jobs,41000.0,2013-11-18 12:00:00,2014-01-17 12:00:00,non-specific,rengineeringjobs.com,non-specific
1,70676070,Senior Civil Infrastructure Engineer,Basingstoke,Matchtech Group plc.,Engineering Jobs,32000.0,2013-06-02 15:00:00,2013-06-16 15:00:00,non-specific,rengineeringjobs.com,non-specific
2,68805528,Highway Technician,Uk,Matchtech Group plc.,Engineering Jobs,19500.0,2012-12-24 15:00:00,2013-01-23 15:00:00,non-specific,rengineeringjobs.com,non-specific
3,71684871,Combustion Engineer Surrey And London upto ****k,London,Randstad,Engineering Jobs,33500.0,2012-10-19 12:00:00,2013-01-17 12:00:00,permanent,rengineeringjobs.com,non-specific
4,67944676,Automation Service Manager,Stockton-On-Tees,Matchtech Group plc.,Engineering Jobs,45000.0,2012-05-03 12:00:00,2012-05-17 12:00:00,non-specific,rengineeringjobs.com,non-specific
5,68805259,Senior/Principal Hardware Engineer,Midlothian,Matchtech Group plc.,Engineering Jobs,36950.0,2012-06-11 15:00:00,2012-06-25 15:00:00,non-specific,rengineeringjobs.com,non-specific
6,69249298,Principal/Senior Mechanical Engineer,Plymouth,Matchtech Group plc.,Engineering Jobs,36290.0,2012-02-09 15:00:00,2012-02-23 15:00:00,non-specific,rengineeringjobs.com,non-specific
7,70331843,EC I Designer /Engineer,Warrington,Milbank,Engineering Jobs,59520.0,2013-08-28 00:00:00,2013-10-27 00:00:00,contract,rengineeringjobs.com,non-specific
8,70675881,Mechanical Design Engineer Peterborough,Peterborough,Randstad,Engineering Jobs,33500.0,2012-03-18 15:00:00,2012-04-17 15:00:00,permanent,rengineeringjobs.com,non-specific
9,70675871,Building Services Supervisor London,London,Randstad,Engineering Jobs,37750.0,2013-05-17 12:00:00,2013-06-16 12:00:00,permanent,rengineeringjobs.com,non-specific


......

#### Finding global key for the data
The global key of the integrated data would be the Id column as there is a unique value each row.

In [30]:
# Code to identify global key
print(df.shape)
df["Id"].value_counts().sum()

(55584, 11)


55584

### 5. Saving the integrated and reshaped data
The last part of the integration process is to export our output data to csv format, named as:
- '\<student\_id\>_dataset_integrated.csv'

In [31]:
# code to save output data
df.to_csv('s3783560_dataset_integrated.csv',index = False)

## Summary of the Assessment Task
Give a short summary and anything you would like to talk about this assessment task here.

To integrate both datasets we first load in the second dataset and fix any conflicts before merging. These could be different data conflicts, where for example, the salary in the second dataset was represent by a value indicating monthly salary which was  different from our global schema. There were also conflicts in schema conflicts where the schema in the second dataset did not match the global schema. These conflicts were resolved accordingly. After merging the two datasets, we then checked for any possible data level conflicts such as duplicates. Any duplicated found where delelted. Finally we saved and extract the integrated dataset to a csv.